### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import astropy.convolution as krn
import scipy.stats as stats
import sys

In [2]:
def makeHeat(screenRes, xPos, yPos):
        xMax = screenRes[0]
        yMax = screenRes[1]
        xMin = 0
        yMin = 0
        kernelPar = 50

        # Input handeling
        xlim = np.logical_and(xPos < xMax, xPos > xMin)
        ylim = np.logical_and(yPos < yMax, yPos > yMin)
        xyLim = np.logical_and(xlim, ylim)
        dataX = xPos[xyLim]
        dataX = np.floor(dataX)
        dataY = yPos[xyLim]
        dataY = np.floor(dataY)

        # initiate map and gauskernel
        gazeMap = np.zeros([int((xMax-xMin)),int((yMax-yMin))])+0.0001
        gausKernel = krn.Gaussian2DKernel(kernelPar)

        # Rescale the position vectors (if xmin or ymin != 0)
        dataX -= xMin
        dataY -= yMin

        # Now extract all the unique positions and number of samples
        xy = np.vstack((dataX, dataY)).T
        uniqueXY, idx, counts = uniqueRows(xy)
        uniqueXY = uniqueXY.astype(int)
        # populate the gazeMap
        gazeMap[uniqueXY[:,0], uniqueXY[:,1]] = counts

        # Convolve the gaze with the gauskernel
        heatMap = np.transpose(krn.convolve_fft(gazeMap,gausKernel))
        heatMap = heatMap/np.max(heatMap)

        return heatMap

def uniqueRows(x):
    y = np.ascontiguousarray(x).view(np.dtype((np.void, x.dtype.itemsize * x.shape[1])))
    _, idx, counts = np.unique(y, return_index=True, return_counts = True)
    uniques = x[idx]
    return uniques, idx, counts


def np_euclidean_distance(y_true, y_pred):

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sqrt(np.sum(np.square(y_pred - y_true), axis=-1))



### Preprocess files and make dataframe
1. 



In [97]:
sys.path.append('./FixationDetection')
from I2MC import runI2MC

# Path to data folders
path_to_folders = 'C:/Users/artem/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_MullerLyer'
# path_to_folders = 'D:/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/online/complete'

# get all folder names
folder_names = os.listdir(path_to_folders)

pp_list = []
num_calib_attemtps = []
for fn in folder_names:
    path = os.path.join(path_to_folders, fn, fn+'_record.csv')       
        
    df = pd.read_csv(path)        

df = df.sort_values('frameNr')
df = df.reset_index(drop=True)
df = df.drop_duplicates(subset=['user_pred_px_x', 'user_pred_px_y'], ignore_index=True)

# a = df[df.event=='target_on']
# r = a[a.target=='right']
# out = r[r.condition=='arrowHeadsOutward']
# inw = r[r.condition=='arrowHeadsInward']

# plt.figure()
# # plt.scatter(out.user_pred_px_x, out.user_pred_px_y)
# # plt.scatter(inw.user_pred_px_x, inw.user_pred_px_y)


fixDF = runI2MC(path, plotData = False)





idx = 0
FixXPos = []
FixYPos = []
FixXPos = np.zeros(df.shape[0])
FixYPos = np.zeros(df.shape[0])
for index, row in df.iterrows():

    
    if idx < fixDF.shape[0]:
        
        if row['sampTime'] >= np.array(fixDF.FixEnd)[idx]:
                idx += 1
        
        if idx < fixDF.shape[0]:
            if row['sampTime'] >= np.array(fixDF.FixStart)[idx] and row['sampTime'] <= np.array(fixDF.FixEnd)[idx]:

                FixXPos[index] = (np.array(fixDF.XPos)[idx])
                FixYPos[index] = (np.array(fixDF.YPos)[idx])
       
   
        
       
        
        
FixXPos = np.array(FixXPos)
FixYPos = np.array(FixYPos)

df['FixXPos'] = FixXPos
df['FixYPos'] = FixYPos



a = df[df.event=='target_on']
a = a[(a.FixXPos!=0) & (a.FixYPos!=0)]
a = a[a.target=='left']

# drop fixations that are not within the target
a = a[a.FixXPos < 550]
# select only start of fixation and drop other frames
a = a.drop_duplicates(subset=['FixXPos', 'FixYPos'], ignore_index=True)
# keep only first fixations and drop corrective fixations
a = a.drop_duplicates(subset=['trialNr'], ignore_index=True)


out = a[a.condition=='arrowHeadsOutward']
inw = a[a.condition=='arrowHeadsInward']


outXmed = out.FixXPos.median()
outYmed = out.FixYPos.median()

inwXmed = inw.FixXPos.median()
inwYmed = inw.FixYPos.median()



plt.figure()
plt.scatter(out.FixXPos, out.FixYPos, c='blue')
plt.scatter(inw.FixXPos, inw.FixYPos, c='orange')
plt.scatter(out.fixationStimX, out.fixationStimY, c='red')
plt.scatter(out.targetX, out.fixationStimY, c='green')

plt.scatter(outXmed, outYmed, c='purple')
plt.scatter(inwXmed, inwYmed, c='yellow')
plt.xlim((0, df.resX[0]))
plt.ylim((0, df.resY[0]))
        

# fixDF.to_csv(self.resultsFile[:-4]+'_fixations.csv', index=False) 


# a = df[df.event=='target_on']
# l = a[a.target=='left']
# out = l[l.condition=='arrowHeadsOutward']
# inw = l[l.condition=='arrowHeadsInward']

# # plt.figure()
# plt.scatter(out.user_pred_px_x, out.user_pred_px_y)
# plt.scatter(inw.user_pred_px_x, inw.user_pred_px_y)
# plt.scatter(out.fixationStimX, out.fixationStimY)
# plt.scatter(out.targetX, out.fixationStimY)









Importing and processing: "C:/Users/artem/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_MullerLyer\2023_06_27_08_39_36\2023_06_27_08_39_36_record.csv"
	Searching for valid interpolation windows
	Replace interpolation windows with Steffen interpolation
	2-Means clustering started for averaged signal
	Determining fixations based on clustering weight mean for averaged signal and separate eyes + 2*std


I2MC took 3.966191053390503s to finish!


C:\Users\artem\anaconda3\envs\default\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\artem\anaconda3\envs\default\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


(0.0, 800.0)

### Plot heatmaps per each condition
Gaze positions for all subjects are combined

In [4]:
# Import fixations into output file




### To do
1. Summarize the incomplete datasets (failed calibration, other reasons)